# This notebook attempts to extract all possible accounts from three *annual* financial reports: the balance sheet, income statement and cash flow statement for cafeF.

### Why this work?
The output of this work will be used for a table mapping a cafeF account to a hard-coded FAD account, which will be stored in our database.

### How is a cafeF account defined?
We define a cafeF account as ...

In [3]:
import pandas as pd
import json
import glob
import re

### Here we define a function to simplify account names

In [2]:
from functions_cafef.fad_crawl_cafef.helpers.processingData import simplifyText

### Define all necessary dictionaries

In [3]:
report_types = {"BS":{},
                "IS":{},
                "CF":{}}
lookup_dict_all = {"BS":{},
                    "IS":{},
                    "CF": {}}

### Process each report type sequentially
Before beginning the code, it's helpful to see what a BS report file looks like:

```
{
    "data": [
        [
            "100",
            "I - TÀI SẢN NGẮN HẠN",
            "186,834,158,952",
            "225,328,146,085",
            "260,649,376,048",
            "218,590,766,332"
        ],
        [
            "110",
            "1. Tiền và các khoản tương đương tiền",
            "9,348,259,155",
            "53,209,096,419",
            "54,853,929,411",
            "13,608,853,476"
        ],
        [
            "111",
            "1.1.Tiền",
            "9,348,259,155",
            "53,209,096,419",
            "54,853,929,411",
            "13,608,853,476"
        ],
        [
            "112",
            "1.2.Các khoản tương đương tiền",
            "",
            "",
            "",
            ""
        ]],
    "periods": [
        "quý 1 2019",
        "quý 2 2019",
        "quý 3 2019",
        "quý 4 2019"
    ]
}
```

In [20]:
### Let's try with BS first
report_type = "BS"
result = []
for file in glob.glob(f'functions_cafef/schemaData/BS_cafef/*{report_type}*.json'):
    with open(file, 'r') as jsonfile:
        try:
            j = json.load(jsonfile)
            for acc in j['data']:
                acc_name = simplifyText(acc[1])
                entry = f'{acc[0]};{acc_name}'
                if entry not in result:
                    result.append(entry)
        except Exception as e:
            print(f'EXCEPTION: {e}')

for a in result:
    print(a)
print(len(result))

100;i tài sản ngắn hạn
110;tiền và các khoản tương đương tiền
111;tiền
112;các khoản tương đương tiền
120;các khoản đầu tư tài chính ngắn hạn
1201;chứng khoán kinh doanh
1202;dự phòng giảm giá chứng khoán kinh doanh
1203;đầu tư nắm giữ đến ngày đáo hạn
130;các khoản phải thu ngắn hạn
131;phải thu khách hàng
132;trả trước cho người bán
133;phải thu nội bộ ngắn hạn
134;phải thu theo tiến độ kế hoạch hợp đồng xây dựng
135;5 phải thu về cho vay ngắn hạn
136;các khoản phải thu khác
139;dự phòng phải thu ngắn hạn khó đòi
1391;8 tài sản thiếu chờ xử lý
140;hàng tồn kho
141;hàng tồn kho
149;dự phòng giảm giá hàng tồn kho
150;tài sản ngắn hạn khác
151;chi phí trả trước ngắn hạn
152;thuế gtgt được khấu trừ
154;thuế và các khoản khác phải thu nhà nước
155;4 giao dịch mua bán lại trái phiếu chính phủ
158;tài sản ngắn hạn khác
200;ii tài sản dài hạn
210;các khoản phải thu dài hạn
211;phải thu dài hạn của khách hàng
212;2 trả trước cho người bán dài hạn
213;vốn kinh doanh ở đơn vị trực thuộc
213;phả

In [6]:
report_types_expressdetails = {"CDKT":{},
                                "KQKD":{},
                                "LC":{}}
lookup_dict_all_expressdetails = {"CDKT":{},
                                    "KQKD":{},
                                    "LC": {}}

with open('functions/schema/bizType_ind_list.json', 'r') as jsonfile:
    biztypes_inds = sorted(json.load(jsonfile))
    jsonfile.close()

for report_type, report_all_accounts in report_types_expressdetails.items():      
    for biztype_ind in biztypes_inds:
        biztype, ind = biztype_ind.split(";")[0], biztype_ind.split(";")[1]
        if biztype_ind not in lookup_dict_all_expressdetails[report_type].keys():
            lookup_dict_all_expressdetails[report_type][biztype_ind] = {}
        if biztype_ind not in report_types_expressdetails[report_type].keys():
            report_types_expressdetails[report_type][biztype_ind] = {}
        for file in glob.glob(f'functions/schemaData/financeInfo/{biztype}_{ind}*{report_type}_Annual*.json'):
            with open(file, 'r') as jsonfile:
                try:
                    j = json.load(jsonfile)
                    for report_fullname, report_content in j[1].items():
                        if report_fullname not in lookup_dict_all_expressdetails[report_type][biztype_ind].keys():
                            lookup_dict_all_expressdetails[report_type][biztype_ind][report_fullname] = {}
                        for content in report_content:
                            acc_id = content['ReportNormID']
                            if acc_id not in lookup_dict_all_expressdetails[report_type][biztype_ind][report_fullname].keys():
                                lookup_dict_all_expressdetails[report_type][biztype_ind][report_fullname][acc_id] = content
                            
                            ### Test if any of the three elements of the entry are ""
#                             acc_n = simplify_text(content['NameEn'])
#                             acc_vi_n = simplify_text(content['Name'])
#                             acc_parent_id = content['ParentReportNormID']
#                             if (acc_n == "" and acc_vi_n == "") or (acc_n == "" and acc_parent_id == "") or (acc_vi_n == "" and acc_parent_id == ""):
#                                 print(file)
#                                 print(f'{acc_n};{acc_vi_n};{acc_parent_id}')
                                
                except Exception as e:
                    print(f'EXCEPTION: {e}')
                    
        for report_fullname, report_content in lookup_dict_all_expressdetails[report_type][biztype_ind].items():
            if report_fullname not in report_all_accounts[biztype_ind].keys():
                report_all_accounts[biztype_ind][report_fullname] = {}
            for acc_id, content in report_content.items():
                acc_n = simplify_text(content['NameEn'])
                acc_vi_n = simplify_text(content['Name'])
                parent_n = simplify_text(lookup_dict_all_expressdetails[report_type][biztype_ind][report_fullname][content['ParentReportNormID']]['NameEn'])
                parent_vi_n = simplify_text(lookup_dict_all_expressdetails[report_type][biztype_ind][report_fullname][content['ParentReportNormID']]['Name'])
                
                entry = f'{acc_n};{parent_n};{acc_vi_n};{parent_vi_n}'
                if entry not in report_all_accounts[biztype_ind][report_fullname].keys():
                    report_all_accounts[biztype_ind][report_fullname][entry] = [acc_id]
                else:
                    report_all_accounts[biztype_ind][report_fullname][entry].append(acc_id)
                    
            for entry, ids in report_all_accounts[biztype_ind][report_fullname].items():
                if len(ids) > 1:
                    print("=== WARNING: one entry has multiple account IDs ===")
                    print(f'ENTRY "{entry}", BIZ TYPE;INDUSTRY "{biztype_ind}", REPORT "{report_type}-{report_fullname}" \n')

            with open(f'functions/schema/{biztype_ind}_{report_type}_{report_fullname}_all_accounts.json', 'w') as writefile:
                json.dump(report_all_accounts[biztype_ind], writefile, ensure_ascii=False, indent=4)

=== WARNING: one entry has multiple account IDs ===
ENTRY ";operating expenses;;cộng chi phí hoạt động 21 33", BIZ TYPE;INDUSTRY "Security Company;Finance and Insurance", REPORT "KQKD-Income Statement" 

=== WARNING: one entry has multiple account IDs ===
ENTRY ";add non cash expenses;;tăng các chi phí phi tiền tệ", BIZ TYPE;INDUSTRY "Security Company;Finance and Insurance", REPORT "LC-CashFlow Indirect" 

=== WARNING: one entry has multiple account IDs ===
ENTRY "cash at bank for securities company activities;add non cash expenses;tiền gửi ngân hàng cho hoạt động ctck;tăng các chi phí phi tiền tệ", BIZ TYPE;INDUSTRY "Security Company;Finance and Insurance", REPORT "LC-CashFlow Indirect" 

=== WARNING: one entry has multiple account IDs ===
ENTRY "cash equivalents;add non cash expenses;các khoản tương đương tiền;tăng các chi phí phi tiền tệ", BIZ TYPE;INDUSTRY "Security Company;Finance and Insurance", REPORT "LC-CashFlow Indirect" 

=== WARNING: one entry has multiple account IDs ===
E